In [ ]:
from mistralai import Mistral
import requests
import numpy as np
import faiss
import os
from dotenv import load_dotenv
from getpass import getpass
load_dotenv()

api_key = os.getenv("MISTRAL_KEY")
client = Mistral(api_key=api_key)

# load data

In [ ]:
f = open('../../data/documentation/rust/doc.md','r')
text = f.read()

In [ ]:
response = requests.get('https://raw.githubusercontent.com/run-llama/llama_index/main/docs/docs/examples/data/paul_graham/paul_graham_essay.txt')
text = response.text


# split document into chuck

In [ ]:
chunk_size = 2048
chunks = [text[i:i + chunk_size] for i in range(0, len(text), chunk_size)]
len(chunks)


## create embedding for each text chunk

In [ ]:
import time
def get_text_embedding(input: str):
    print(input)
    embeddings_batch_response = client.embeddings.create(
          model="mistral-embed",
          inputs=input
      )
    time.sleep(2)
    print("Embeddings created------------------------------------")
    
    return embeddings_batch_response.data[0].embedding
text_embeddings = np.array([get_text_embedding(chunk) for chunk in chunks])

In [ ]:
import faiss

d = text_embeddings.shape[1]
index = faiss.IndexFlatL2(d)
index.add(text_embeddings)

In [ ]:
question = "What were the two main things the author worked on before college?"
question_embeddings = np.array([get_text_embedding(question)])

In [ ]:
D, I = index.search(question_embeddings, k=2) # distance, index
retrieved_chunk = [chunks[i] for i in I.tolist()[0]]

In [ ]:
prompt = f"""
Context information is below.
---------------------
{retrieved_chunk}
---------------------
Given the context information and not prior knowledge, answer the query.
Query: {question}
Answer:
"""

In [ ]:
def run_mistral(user_message, model="mistral-large-latest"):
    messages = [
        {
            "role": "user", "content": user_message
        }
    ]
    chat_response = client.chat.complete(
        model=model,
        messages=messages
    )
    return (chat_response.choices[0].message.content)

run_mistral(prompt)